<a href="https://colab.research.google.com/github/arnavm30/skateboard_deck_generation/blob/main/StyleGAN2_ADA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA

StyleGAN2-ADA only works with Tensorflow 1. 

In [ ]:
%tensorflow_version 1.x

Shows the assigned GPU

In [ ]:
!nvidia-smi

# Setting up Drive

Mount Drive to the Colab notebook so that the StyleGAN2-ADA repo can be downloaded into a Drive folder.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

If repo already installed, skip the installation process and cd into the repo. If not installed, install it.

In [ ]:
import os
if os.path.isdir('/content/drive/My Drive/StyleGAN2-ADA_colab'):
    %cd '/content/drive/My Drive/StyleGAN2-ADA_colab/stylegan2-ada'
else:
    %cd '/content/drive/My Drive/'
    !mkdir StyleGAN2-ADA_colab
    %cd StyleGAN2-ADA_colab
    !git clone https://github.com/NVlabs/stylegan2-ada
    %cd stylegan2-ada
    !mkdir datasets

# Preparing Dataset

*This section only needs to be run the first time training to download the dataset into Drive*

Unzipping zipped dataset uploaded directly to colab

In [ ]:
zip_path = '/content/decks.zip'
outdir = '/content/'
!unzip {zip_path} -d {outdir}

Converting zipped dataset to a TFRecord, the format StyleGAN2-ADA uses.

In [ ]:
dataset = './datasets/decks'
dataset_path = '/content/decks'

!python dataset_tool.py create_from_images {dataset} {dataset_path}

# Training Model

Setting arguments and training model.

In [ ]:
# for first time training, set to 'ffhq512' for transfer learning so that convergence is faster
# after, set to path of latest .pkl file to resume where previous training left off
resume_from = '/content/drive/MyDrive/StyleGAN2-ADA_colab/stylegan2-ada/results/00007-decks-mirror-auto1-bg-resumecustom/network-snapshot-000240.pkl'

dataset = './datasets/decks'

# augmentations: (b)lit, (g)eom
augs = 'bg'

# images mirrored left to right
mirrored = True 

# model generate samples and a .pkl file with model weights every 4 ticks
snapshot_count = 4 

# no metrics since quality of output is kind of subjective and running metrics adds a lot of time
metrics = None 

outdir = './results'

# training
!python train.py  --resume {resume_from} --data {dataset} --augpipe {augs} --mirror {mirrored} --snap {snapshot_count} --metrics {metrics} --outdir {outdir}

# Testing Model

Downloading model from Drive

In [ ]:
id = '117ZORa1dPkpkMxVKhugPmcioXFKL4zFx' #google id of saved model in .pkl format
outdir = '/content/network.pkl'
!gdown --id {id} -O {outdir}

opensimplex is needed for programs used to test the model

In [ ]:
!pip install opensimplex

 ### Generating Images

`--seeds` chooses random seeds from the model; each seed generates one random array

In [ ]:
network = '/content/network.pkl'
seeds='1-40'
outdir = '/content/drive/MyDrive/StyleGAN2-ADA_colab/stylegan2-ada//testing/generate'

!python generate.py generate-images --network {network} --seeds {seeds} --outdir {outdir}

### Interpolation
generating small changes to a vector to make it appear animated between each frame

`--walk-type`: type of interpolation (line or noise loop) and latent space (z space or w space)

`--frames`: amount of frames to make

`--trunc`: number that relates to level of realisticness of the the ouptut. 0.0-1.0 produce realistic results

###### Line interpolation
linear path from one seed to another in w or z space.

In [ ]:
network = '/content/network.pkl'
seeds = '13,19,94,133,67,22,13'
walk = 'line-w'
outdir = '/content/drive/MyDrive/StyleGAN2-ADA_colab/stylegan2-ada//testing/line-w'

!python generate.py generate-latent-walk --network {network} --walk-type {walk} --seeds {seeds} --outdir {outdir}

###### Noise loop interpolation
random path through z space

`--diameter`: number that controls the radius of the loop (small means less diverse sample, large means more diverse)

`--start_seed`: starting place in z space

In [ ]:
network = '/content/network.pkl'
walk = 'noiseloop'
diameter = '1.5'
start_seed = '0'
outdir = '/content/drive/MyDrive/StyleGAN2-ADA_colab/stylegan2-ada/testing/noiseloop'

!python generate.py generate-latent-walk --network {network} --walk-type {walk} --start_seed {start_seed} --diameter {diameter} --outdir {outdir}

### Projection
taking an image outside the model and finding the closest representation generated by the model. 


In [ ]:
%cd /content/drive/My Drive/StyleGAN2-ADA_colab/stylegan2-ada/
network = '/content/network.pkl'
target = '/content/ccs_40.png'
outdir = '/content/drive/MyDrive/StyleGAN2-ADA_colab/stylegan2-ada/testing/projection'

!python projector.py  --network {network} --target {target} --outdir {outdir}

# Displaying Results

This cell is used to display images generated from running tests

In [ ]:
from IPython.display import Image, display
import glob, os
images = '/content/drive/MyDrive/StyleGAN2-ADA_colab/stylegan2-ada/testing/projection'
os.chdir(images)
for image_name in glob.glob('*.png'):
  display(Image(filename=image_name, width=512))